In [1]:
!export PYSPARK_PYTHON=python3.6.9

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D
from tensorflow.keras.layers import MaxPooling2D, Dropout,Flatten
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding
import subprocess
import tensorflow as tf
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from nltk.corpus import brown
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [3]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row
from pyspark.streaming.kafka import KafkaUtils
import pyspark.sql.functions as f
import json
import re
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
import sys
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import from_json
from pyspark.sql.functions import lit

import pickle
import tensorflow.keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import zeros
from keras.models import model_from_json
from tensorflow import keras

# import tensorflow.keras
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from numpy import zeros
# from tensorflow.keras.models import model_from_json

Using TensorFlow backend.


In [4]:
print(tf.__version__)

2.4.4


In [5]:
spark.sparkContext.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.executor.instances', '3'),
 ('spark.sql.execution.arrow.enabled', 'true'),
 ('spark.driver.memory', '4g'),
 ('spark.executor.cores', '5'),
 ('spark.executor.extraJavaOptions', '-XX:+UseG1GC'),
 ('spark.driver.extraJavaOptions', '-XX:+UseG1GC'),
 ('spark.driver.host', 'kafka1'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.yarn.archive', 'hdfs:///user/spark/conf/spark-libs.jar'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.sql.shuffle.partitions', '200'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.filters',
  'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'),
 ('spark.kryoserializer.buffer.max', '2047'),
 ('spark.executor.memoryOverhead', '1g'),
 ('spark.driver.memoryOverhead', '1g'),
 ('spark.executor.memory', '4g'),
 ('spark.executor.id', 'driver'),
 ('spark.history.fs.logDirectory', 'file:///root/spark/eventLog'),
 ('spark.default.parallelism', '163'),
 ('spark.app.name', 

In [6]:
# MongoDB 데이터 불러오기
import pymongo
from pymongo import MongoClient

connection = MongoClient('mongodb://117.17.189.202:27027')
tweet = connection.test
kafka_tweet = tweet.kafka_tweet

In [7]:
import pandas as pd
doc = kafka_tweet.find({})
dfdf =  pd.DataFrame(list(doc))
dfdf['text'][:10]

0    RT @nanox_finance: #Airdrop Alert 🚨 \n\n@nanox...
1                    */inamin HAHAHAHAHAHAHAHAHAHAHAHA
2    4.  hai yang mau sewa zoom, unlock chegg,cek t...
3               @AEDi32 para sa concert HAHAHAHAHAHAHA
4    RT @KariOrenday: Lo de hoy es culpa de dos, Sa...
5    RT @BTS_jp_official: #BTS ARTIST-MADE COLLECTI...
6    RT @esbingcali: Para sa akin, “WHAT?” ang titl...
7    Figueiredo enfin il récupérer sa ceinture mdrr...
8    @kisshmtc_16229 ご応募ありがとうございました❗\n結果は\n\n残念…はずれ...
9    RT @kep1er_worrld: [ADMIN] 220123 - Kep1ians w...
Name: text, dtype: object

In [6]:
# 임시코드
csi_pos_neg = spark.read.csv("hdfs:///user/spark/datafile/csiposneg.csv", header=True)
csi_pos_neg = csi_pos_neg.withColumn("label",col("label").cast("integer"))

In [7]:
f = open('/root/keywordset.txt', 'r', encoding='utf-8')
keyfile = f.readline()
keyword_list = list()
keywords = keyfile.split(',')
for keyword in keywords:
    keyword_list.append(keyword.strip())

In [61]:
df1 = dfdf[:1101600][['text']]
df1 = spark.createDataFrame(df1)

/root/spark/python/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  An error occurred while calling z:org.apache.spark.sql.api.python.PythonSQLUtils.readArrowStreamFromFile.
: java.lang.IllegalArgumentException
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:334)
	at org.apache.arrow.vector.ipc.message.MessageSerializer.readMessage(MessageSerializer.java:543)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anon$3.readNextBatch(ArrowConverters.scala:243)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anon$3.<init>(ArrowConverters.scala:229)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$.getBatchesFromStream(ArrowConverters.scala:228)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anonfun$readArrowStreamFromFile$2.apply(ArrowConverters.scala:216)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anonfun

In [10]:
df2 = dfdf[1101600:2203200][['text']]
df2 = spark.createDataFrame(df2)

/root/spark/python/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  An error occurred while calling z:org.apache.spark.sql.api.python.PythonSQLUtils.readArrowStreamFromFile.
: java.lang.IllegalArgumentException
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:334)
	at org.apache.arrow.vector.ipc.message.MessageSerializer.readMessage(MessageSerializer.java:543)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anon$3.readNextBatch(ArrowConverters.scala:243)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anon$3.<init>(ArrowConverters.scala:229)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$.getBatchesFromStream(ArrowConverters.scala:228)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anonfun$readArrowStreamFromFile$2.apply(ArrowConverters.scala:216)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anonfun

In [8]:
t1 = csi_pos_neg.collect()[:24939]
t2 = csi_pos_neg.collect()[24939:49878]
t3 = csi_pos_neg.collect()[49878:89756]
t4 = csi_pos_neg.collect()[89756:99756]

t5 = csi_pos_neg.collect()[99756:124695]
t6 = csi_pos_neg.collect()[124695:149634]
t7 = csi_pos_neg.collect()[149634:189512]
t8 = csi_pos_neg.collect()[189512:199512]

t1 = spark.createDataFrame(t1)
t2 = spark.createDataFrame(t2)
t3 = spark.createDataFrame(t3)
t4 = spark.createDataFrame(t4)

t5 = spark.createDataFrame(t5)
t6 = spark.createDataFrame(t6)
t7 = spark.createDataFrame(t7)
t8 = spark.createDataFrame(t8)

first_df = t1.union(t5).union(t2).union(t6) #.union(t3).union(t7) # 179512 
first_df_df = t3.union(t7)
second_df = t4.union(t8) # 20000 -> unrevealed data for all periods
first_df = first_df.drop(col('_c0'))
first_df_df = first_df_df.drop(col('_c0'))
second_df = second_df.drop(col('_c0'))

In [ ]:
################### 2번째 time window##########################

In [12]:
# compare_and_drop for semi-supervised learning
df = spark.read.csv("hdfs:///user/spark/datafile/first_df.csv", header=True)
df = df.toPandas()
df = df.astype({'text':'string'})
df = df.astype({'label':'double'})

df_1 = df[df['label']==1.0]
length = len(df_1)
df_2 = df[df['label']==0.0][:length]

In [13]:
df_1_train = df_1[:-10000]
df_1_test = df_1[-10000:]
df_2_train = df_2[:-10000]
df_2_test = df_2[-10000:]

In [14]:
plus_df = pd.concat([df_1_test, df_2_test])
plus_df.reset_index(inplace=True, drop=True)
plus_df.drop('_c0', axis=1, inplace=True)


In [15]:
print(len(plus_df))
plus_df = spark.createDataFrame(plus_df)

20000


/root/spark/python/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Cannot specify a mask or a size when passing an object that is converted with the __arrow_array__ protocol.
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


In [16]:
second_df = second_df.drop(col('_c0'))
second_df.show(3)

+--------------------+-----+
|                text|label|
+--------------------+-----+
|jvndboracle peopl...|    1|
|lazyconsultant wi...|    1|
|avec peuttre un p...|    1|
+--------------------+-----+
only showing top 3 rows



In [17]:
# second_df['label'].value_counts()

In [18]:
second_df = second_df.union(plus_df) # 4만개로 늘어남

In [19]:
first_df = pd.concat([df_1_train, df_2_train])
first_df.reset_index(inplace=True, drop=True)
first_df

,_c0,text,label
0,5,figueiredo enfin il rcuprer sa ceinture mdrr i...,1.0
1,19,mercedesamgf in but on other sport,1.0
2,27,rt shezzcryptogem the new standard for home re...,1.0
3,33,im zizzy and this is my friend pony,1.0
4,39,i love that this has already its crusty gif tr...,1.0
...,...,...,...
202209,128349,rt zorochi hr date hengsurpriselink ht,0.0
202210,128350,rt siphillipssport i am told that the players ...,0.0
202211,128351,anwdverpkbczx twitter,0.0
202212,128352,rt gppulipaka crossadversarial learning for mo...,0.0


In [20]:
first_df.drop('_c0', axis=1, inplace=True)
first_df.dtypes
first_df = spark.createDataFrame(first_df)

/root/spark/python/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Cannot specify a mask or a size when passing an object that is converted with the __arrow_array__ protocol.
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


In [29]:
class voc:
    staticList = list() # 현재는 계속해서 incremental하게 사이즈가 커지고 있다

class word_index:
    staticDict = dict() # 현재는 계속해서 incremental하게 사이즈가 커지고 있다

class vocabulary:
    staticList = list()
    
class aaa:
    staticList = list()

In [13]:
type(word_index.staticDict)

dict

In [58]:
with open('2_voc.pkl','wb') as f: 
    pickle.dump(voc.staticList,f)
    
with open('2_word_index.pkl','wb') as f: 
    pickle.dump(word_index.staticDict,f)
    
with open('2_vocabulary.pkl','wb') as f: 
    pickle.dump(vocabulary.staticList,f)

In [12]:
with open('1_voc.pkl','rb') as f: 
    voc.staticList = pickle.load(f)
    
with open('1_word_index.pkl','rb') as f: 
    word_index.staticDict = pickle.load(f)
    
with open('1_vocabulary.pkl','rb') as f: 
    vocabulary.staticList = pickle.load(f)

In [30]:
#계속해서 선언해주기
class a:
    length = len(voc.staticList) # 현재는 계속해서 incremental하게 사이즈가 커지고 있다

class b:
    length = len(word_index.staticDict) # 현재는 계속해서 incremental하게 사이즈가 커지고 있다

class c:
    length = len(vocabulary.staticList)

In [10]:
word_index.staticDict

{}

In [31]:
len(voc.staticList)

0

In [32]:
a.length

0

In [ ]:
from pyspark.ml import Transformer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from typing import Dict
from pyspark.sql import DataFrame

class TextToSequence(Transformer):
    vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
    
    def __init__(self):
        super(TextToSequence, self).__init__()
        
    def _transform(self, df: DataFrame):
        # 우리가 해야할 일은 dataframe을 순서통일해서 text와 label 리스트로 나누는것
        samples = [' '.join(sample) for sample in df.select("filtered_words").rdd.flatMap(lambda x: x).collect()]
        
        labels = df.select("label").rdd.flatMap(lambda x: x).collect()
        words = df.select("filtered_words").rdd.flatMap(lambda x: x).collect()
        # 200의 길이로 상위 2만개의 토큰만 반영
        text_ds = tf.data.Dataset.from_tensor_slices(samples).batch(128)
        self.vectorizer.adapt(text_ds)
        
        temp = voc.staticList
        print(len(temp))
        # 엄데이트 하는 식으로 구현 동작 바꾸기!
        voc.staticList = list(set(voc.staticList + self.vectorizer.get_vocabulary()))
        
        aa = list(set(self.vectorizer.get_vocabulary())-set(temp))
        print(len(aa),"!")
        
        post_length = len(voc.staticList)
        vocabulary.staticList = np.concatenate(words).tolist()
        word_index.staticDict.update(dict(zip(aa, range(a.length, a.length+len(aa)))))
        
        x_train = self.vectorizer(np.array([[s] for s in samples])).numpy()
        zip_array = list(zip(samples, labels, x_train.tolist()))
        rdd = sc.parallelize(zip_array, 306) #, numSlices=306
        fdf = rdd.toDF(['text','label','feature'])
        
        return fdf

In [34]:
[' '.join(sample) for sample in preprocessed.select("filtered_words").rdd.flatMap(lambda x: x).collect()]

AnalysisException: "cannot resolve '`filtered_words`' given input columns: [text, label, feature, label_index];;\n'Project ['filtered_words]\n+- Project [text#138, label#139L, feature#140, UDF(cast(label#139L as string)) AS label_index#144]\n   +- LogicalRDD [text#138, label#139L, feature#140], false\n"

In [13]:
## old version
from pyspark.ml import Transformer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from typing import Dict
from pyspark.sql import DataFrame

class TextToSequence(Transformer):
    vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
    
    def __init__(self):
        super(TextToSequence, self).__init__()
        
    def _transform(self, df: DataFrame):
        # 우리가 해야할 일은 dataframe을 순서통일해서 text와 label 리스트로 나누는것
        samples = df.select("text").rdd.flatMap(lambda x: x).collect()
        labels = df.select("label").rdd.flatMap(lambda x: x).collect()
        words = df.select("filtered_words").rdd.flatMap(lambda x: x).collect()
        # 200의 길이로 상위 2만개의 토큰만 반영
        #vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
        text_ds = tf.data.Dataset.from_tensor_slices(samples).batch(128)
        self.vectorizer.adapt(text_ds)
        
        temp = voc.staticList
        print(len(temp))
        # 엄데이트 하는 식으로 구현 동작 바꾸기!
        voc.staticList = list(set(voc.staticList + self.vectorizer.get_vocabulary()))
        
        aa = list(set(self.vectorizer.get_vocabulary())-set(temp))
        aaa.staticList = list(set(aaa.staticList + aa))
        print(len(aa),"!")
        
        post_length = len(voc.staticList)
        vocabulary.staticList = np.concatenate(words).tolist()
        #np.array(words).flatten()# samples에 존재하는 데이터 리스트 (중복허용)
        #word_index.staticDict = dict(zip(voc.staticList, range(len(voc.staticList))))
        word_index.staticDict.update(dict(zip(aa, range(a.length, a.length+len(aa)))))
        
        x_train = self.vectorizer(np.array([[s] for s in samples])).numpy()
        zip_array = list(zip(samples, labels, x_train.tolist()))
        rdd = sc.parallelize(zip_array, 306) #, numSlices=306
        fdf = rdd.toDF(['text','label','feature'])
        
        return fdf

In [14]:
# get_tokens 함수과 같은 용도
stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
# get_lemma 함수와 같은 용도
stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')
stage_3 = TextToSequence()
stage_4 = StringIndexer(inputCol='label', outputCol='label_index')
#label_df = label_str_index.fit(df_with_vectors).transform(df_with_vectors)
transformer_pipeline = Pipeline(stages=[stage_1, stage_2, stage_3, stage_4])
preprocessing = transformer_pipeline.fit(first_df)
preprocessed = preprocessing.transform(first_df)

0
20000 !
20000
0 !


In [35]:
word_index.staticDict

{'': 0,
 'mobile': 1,
 'filename': 2,
 'circle': 3,
 'fingerprints': 4,
 'cardiology': 5,
 'radeon': 6,
 'review': 7,
 'arte': 8,
 'grilled': 9,
 'uba': 10,
 'lazysunday': 11,
 'arti': 12,
 'wired': 13,
 'jug': 14,
 'evolution': 15,
 'rex': 16,
 'why': 17,
 'kale': 18,
 'brandon': 19,
 'cooper': 20,
 'cranberry': 21,
 'hi': 22,
 'voltaire': 23,
 'hells': 24,
 'ela': 25,
 'gu': 26,
 'networkcentric': 27,
 'uxbridge': 28,
 'crop': 29,
 'spacex': 30,
 'avengers': 31,
 'mart': 32,
 'errors': 33,
 'comptia': 34,
 'dread': 35,
 'flir': 36,
 'cissp': 37,
 'siemens': 38,
 'mistakes': 39,
 'symsasa': 40,
 'shepherd': 41,
 'general': 42,
 'warmed': 43,
 'happyprinceday': 44,
 'presbyterian': 45,
 'williams': 46,
 'masks': 47,
 'allah': 48,
 'herb': 49,
 'troubleshooting': 50,
 'resuming': 51,
 'pull': 52,
 'northern': 53,
 'chased': 54,
 'blackmail': 55,
 'plea': 56,
 'hybrid': 57,
 'ginger': 58,
 'assault': 59,
 'ponds': 60,
 'machine': 61,
 'redirecting': 62,
 'coherent': 63,
 'elevated': 64,


In [36]:
len(voc.staticList) # 20000, 31732  ##31701, 32512

25566

In [37]:
len(word_index.staticDict.keys())

25566

In [38]:
len(set(vocabulary.staticList))

125630

In [28]:
vocabulary.staticList[:10]

['rt',
 'chaescutie',
 'wtslfbtwice',
 'assorted',
 'photocards',
 'mopgcash',
 'set',
 'modshopee',
 'coreplydm',
 'claim']

In [29]:
preprocessed.columns

['text', 'label', 'feature', 'label_index']

In [14]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
df_with_vectors = preprocessed.select(
    preprocessed["text"],
    preprocessed["label_index"],
    list_to_vector_udf(preprocessed["feature"]).alias("feature")
)

In [18]:
df_with_vectors.columns

['text', 'label_index', 'feature']

In [ ]:
#############################################################################

In [18]:
import csv
import numpy as np
from collections import Counter
from nltk.corpus import brown
from mittens import GloVe, Mittens
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
import pickle

def glove2dict(glove_filename):
    with open(glove_filename, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=' ',quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed

glove_path = "/root/spark/glove.6B.100d.txt"
pre_glove = glove2dict(glove_path)

In [19]:
len(pre_glove)

400000

In [40]:
#두번째 time window 이상부터 각 time window에 맞추어서 repo_glove_N
# load the pre-trained Glove weight
f = open("repo_glove_1.pkl","rb")
pre_glove = pickle.load(f)

In [41]:
len(pre_glove)

400621

In [42]:
len(vocabulary.staticList)

732787

In [25]:
sw = list(_stop_words.ENGLISH_STOP_WORDS)
# 새로운 데이터에서의 token
brown_data = vocabulary.staticList #[:200000]
brown_data

['threatmeter',
 'hacked',
 'emails',
 'san',
 'francisco',
 'muni',
 'rail',
 'system',
 'hacker',
 'reveals',
 'clues',
 'identity',
 'tactics',
 'kr',
 'fin',
 'first',
 'android',
 'malware',
 'targeting',
 'pcs',
 'uncovered',
 'fbxtynm',
 'adobe',
 'fixes',
 'six',
 'code',
 'execution',
 'bugs',
 'flash',
 'mokgxr',
 'scienceporn',
 'vacuum',
 'guess',
 'riskware',
 'hmoyfzb',
 'fbjaz',
 'rt',
 'exploithunt',
 'wzemae',
 'data',
 'protection',
 'fusion',
 'itsec',
 'databreach',
 'news',
 'wzenei',
 'webroot',
 'spamvertised',
 'lost',
 'message',
 'facebook',
 'campaign',
 'leads',
 'pharmaceutical',
 'scams',
 'xunah',
 'cybersecurity',
 'hacker',
 'privacy',
 'big',
 'data',
 'rmge',
 'postsnag',
 'gingcredsfromlockedmachines',
 'ddos',
 'mvrzcc',
 'top',
 'actions',
 'runtime',
 'application',
 'protection',
 'fbodttge',
 'watch',
 'aqvqvy',
 'threatmeter',
 'facebook',
 'algorithm',
 'watching',
 'loetua',
 'product',
 'model',
 'v',
 'product',
 'name',
 'forensic',
 'ultr

In [26]:
# stopwords 제외한 token
brown_nonstop = [token.lower() for token in brown_data if (token.lower() not in sw)]
len(brown_nonstop)

1777350

In [28]:
kkk = brown.words()[:200000]
print(len(set(kkk)))
print(kkk[:100])

22364
['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]


In [20]:
# 기존에 pre_glove에서 없던 새로운 토큰들
oov = [token for token in brown_nonstop if token not in pre_glove.keys()]
len(oov)

3112

In [21]:
len(set(oov))

3112

In [22]:
def get_rareoov(xdict, val):
    return [k for (k,v) in Counter(xdict).items() if v<=val]
# frequency가 10개 이하인 데이터 집합
oov_rare = get_rareoov(oov, 1) # 처음에만 200으로 설정하고 그 후로는 500으로 설정

In [23]:
len(oov_rare)

3112

In [24]:
# frequency가 10개 미만인 데이터 제외한 새로운 토큰들
corp_vocab = list(set(oov) - set(oov_rare))
len(corp_vocab)

0

In [50]:
# oov_rare에서 keyword_list내의 token을 제거하고
oov_rare = list(set(oov_rare) - set(keyword_list))
len(oov_rare)

84446

In [51]:
len(aaa.staticList)

25566

In [52]:
len(aaa.staticList[0]) + 650

650

In [53]:
# corp_vocab에 keyword_list내의 token을 append해야함
corp_vocab = list(set(corp_vocab + keyword_list)) ## 여기에corp_vocab대신 aa(즉 voc.staticList에 추가된 단어도 추가 고려,,)
len(corp_vocab)

650

In [36]:
# 새로운 데이터중에서 frequency가 10미만 데이터를 제외하고 새로운 토큰들
# time complexity가 너무 커서 실행하지 못함
# 따라서 그냥 brown_doc에 brown_nonstop만 넘겨줌..
brown_tokens = [token for token in brown_nonstop if token not in oov_rare]

KeyboardInterrupt: 

In [54]:
brown_doc = [' '.join(brown_nonstop)]

In [55]:
import csv
import numpy as np
from collections import Counter
from nltk.corpus import brown
from mittens import GloVe, Mittens
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
import pickle
# corp_vocab = list(set(oov))
# brown_doc = [' '.join(brown_nonstop)]

cv = CountVectorizer(ngram_range=(1,1), vocabulary=corp_vocab)
X = cv.fit_transform(brown_doc)
Xc = (X.T * X)
Xc.setdiag(0)
coocc_ar = Xc.toarray()

mittens_model = Mittens(n=100, max_iter=10000, learning_rate=0.001) # small learning rate should be offered 0.0001

new_embeddings = mittens_model.fit(
    coocc_ar,
    vocab=corp_vocab,
    initial_embedding_dict= pre_glove)

newglove = dict(zip(corp_vocab, new_embeddings))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
Iteration 10000: loss: 60.98265838623047

In [56]:
pre_glove.update(newglove)

f = open("repo_glove_2.pkl","wb")
pickle.dump(pre_glove, f)
print('success')
f.close()

success


In [36]:
len(newglove.keys())

657

In [57]:
len(pre_glove.keys())

400621

# 예측 수행

In [19]:
df_with_vectors.show(5)

+--------------------+-----------+--------------------+
|                text|label_index|             feature|
+--------------------+-----------+--------------------+
|threatmeter hacke...|        1.0|[176.0,319.0,1385...|
|first android mal...|        1.0|[115.0,128.0,82.0...|
|adobe fixes six c...|        1.0|[562.0,1449.0,220...|
| scienceporn  in ...|        1.0|[1.0,4.0,5.0,1438...|
|riskware hmoyfzb ...|        1.0|[1.0,1.0,1.0,0.0,...|
+--------------------+-----------+--------------------+
only showing top 5 rows



In [20]:
# load the pre-trained Glove weight
f = open("repo_glove_1.pkl","rb")
newglove = pickle.load(f)

num_tokens = len(voc.staticList) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.staticDict.items():
    embedding_vector = newglove.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 16486 words (3514 misses)


In [21]:
len(newglove.keys())

400621

In [22]:
from tensorflow.keras.layers import Embedding
from tensorflow import keras
nb_classes = 2

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [23]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu", name='f1')(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu", name='f2')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu", name='f3')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu", name='d1')(x)
preds = layers.Dense(2, activation="softmax", name='d2')(x)
pretrained = keras.Model(int_sequences_input, preds, name="pretrained_model")
pretrained.summary()

Model: "pretrained_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         2000200   
_________________________________________________________________
f1 (Conv1D)                  (None, None, 128)         64128     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 128)         0         
_________________________________________________________________
f2 (Conv1D)                  (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
f3 (Conv1D)                  (None, None, 128)    

In [24]:
pretrained.compile(
    loss="categorical_crossentropy", optimizer="sgd", metrics=["acc"]
)

## fine-tune

In [43]:
# load the pre-trained Glove weight
f = open("repo_glove_2.pkl","rb")
newglove = pickle.load(f)

num_tokens = len(voc.staticList) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.staticDict.items():
    embedding_vector = newglove.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 20063 words (5503 misses)


In [44]:
from tensorflow.keras.layers import Embedding
from tensorflow import keras
nb_classes = 2

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    name = "e1" # 계속 바꿔줘야해
)

In [22]:
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu", name='f1')(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu", name='f2')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu", name='f3')(x)
x = layers.GlobalMaxPooling1D()(x)
pretrained_model = keras.Model(int_sequences_input, x, name="pretrained")

model = keras.Sequential([pretrained_model, layers.Dense(128, activation="relu", name='d1'), layers.Dense(2, activation="softmax", name='d2')])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
pretrained (Functional)      (None, 128)               2785024   
_________________________________________________________________
d1 (Dense)                   (None, 128)               16512     
_________________________________________________________________
d2 (Dense)                   (None, 2)                 258       
Total params: 2,801,794
Trainable params: 244,994
Non-trainable params: 2,556,800
_________________________________________________________________


In [23]:
pretrained_model.load_weights("pretrained_ckpt")

In [112]:
pretrained_model.layers

In [24]:
pretrained_model.layers[1].weights

[<tf.Variable 'e1/embeddings:0' shape=(25568, 100) dtype=float32, numpy=
 array([[ 0.32015198,  0.32985097, -0.63490605, ...,  0.605566  ,
         -1.7903564 ,  1.261064  ],
        [-1.0016309 ,  2.0705097 ,  0.06739932, ..., -1.5806402 ,
         -0.74558794,  0.5540653 ],
        [ 0.828892  ,  0.42481294, -1.5089525 , ...,  0.8579917 ,
          1.1408814 , -0.8615934 ],
        ...,
        [ 0.7850558 ,  2.0836573 , -1.2791005 , ..., -0.15426576,
         -1.610927  ,  1.0542847 ],
        [ 0.6916777 ,  0.4460445 , -0.70862865, ..., -1.1327271 ,
         -0.97815156,  1.7759557 ],
        [-1.8696328 ,  1.4004666 , -0.22077304, ..., -0.2562677 ,
         -0.12459616,  0.33991626]], dtype=float32)>]

In [127]:
pretrained_model.layers[9].weights

[<tf.Variable 'd2/kernel:0' shape=(128, 2) dtype=float32, numpy=
 array([[ 0.568587  , -0.5979264 ],
        [-0.4200679 , -0.3741734 ],
        [-0.80700105, -2.3267548 ],
        [-0.6535369 ,  0.58927923],
        [-2.9724457 , -0.10927045],
        [-0.2552872 , -0.02338264],
        [-0.01662377, -0.93165255],
        [-0.3466121 ,  0.6077571 ],
        [ 1.0392745 ,  1.8214877 ],
        [ 0.151336  ,  0.68245864],
        [ 0.18562372,  1.4929657 ],
        [ 0.39345235, -0.62674046],
        [-0.10179545, -1.6648691 ],
        [-0.742026  , -0.5744636 ],
        [ 0.26150528, -1.21723   ],
        [ 0.861037  ,  1.450817  ],
        [-0.13965195, -2.2885194 ],
        [-0.65240747,  0.41471517],
        [-1.4281089 , -1.248416  ],
        [-0.8578196 ,  1.0589912 ],
        [ 0.8788392 ,  0.6632096 ],
        [ 0.78547144, -0.11804313],
        [ 1.0712187 ,  0.03534868],
        [ 1.1237779 , -1.1491095 ],
        [ 2.7552938 ,  0.5297781 ],
        [-0.53636134, -1.999765  ],

In [90]:
pretrained_model.layers[1].set_weights([embedding_matrix])
pretrained_model.layers[1].trainable = False

In [25]:
pretrained_model.layers

In [26]:
for layer in pretrained_model.layers[:-2]: 
    layer.trainable = False

In [23]:
# Check that all of the pretrained weights have been loaded.
for a, b in zip(pretrained.weights, pretrained_model.weights):
    np.testing.assert_allclose(a.numpy(), b.numpy())

NameError: name 'pretrained' is not defined

In [27]:
print('model를 동결하기 전 훈련되는 가중치의 수:',
     len(pretrained_model.trainable_weights))
print('model를 동결하기 전 훈련되는 가중치의 수:',
     len(model.trainable_weights))
#pretrained_model.trainable = False
print('model를 동결한 후 훈련되는 가중치의 수:',
     len(pretrained_model.trainable_weights))
print('model를 동결한 후 훈련되는 가중치의 수:',
     len(model.trainable_weights))

model를 동결하기 전 훈련되는 가중치의 수: 2
model를 동결하기 전 훈련되는 가중치의 수: 6
model를 동결한 후 훈련되는 가중치의 수: 2
model를 동결한 후 훈련되는 가중치의 수: 6


In [31]:
# from tensorflow.keras import layers

# int_sequences_input = keras.Input(shape=(None,), dtype="int64")
# embedded_sequences = embedding_layer(int_sequences_input)
# x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
# x = layers.MaxPooling1D(5)(x)
# x = layers.Conv1D(128, 5, activation="relu")(x)
# x = layers.MaxPooling1D(5)(x)
# x = layers.Conv1D(128, 5, activation="relu")(x)
# x = layers.GlobalMaxPooling1D()(x)
# pretrained_model = keras.Model(inputs=int_sequences_input, outputs=x, name="pretrained")

# # Sequential example:
# model = keras.Sequential([pretrained_model, layers.Dense(128, activation="relu"), layers.Dense(2, activation="softmax")])
# model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
pretrained (Functional)      (None, 128)               2785024   
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 258       
Total params: 2,801,794
Trainable params: 244,994
Non-trainable params: 2,556,800
_________________________________________________________________


In [39]:
# for w in model.weights:
#     print(w)

<tf.Variable 'embed/embeddings:0' shape=(25568, 100) dtype=float32, numpy=
array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.13277 , -0.11802 , -0.063487, ...,  0.10199 ,  1.0979  ,
        -0.17951 ],
       [-0.016233,  0.77261 , -0.052126, ...,  0.067194, -0.2468  ,
        -0.12894 ],
       ...,
       [-0.75831 ,  0.42915 , -0.040031, ...,  0.11268 , -0.088691,
        -0.20963 ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ]], dtype=float32)>
<tf.Variable 'conv1d_6/kernel:0' shape=(5, 100, 128) dtype=float32, numpy=
array([[[ 0.02555508, -0.03761939, -0.01906643, ...,  0.03926139,
          0.04262729, -0.0114852 ],
        [ 0.06523345,  0.06422513,  0.04403574, ..., -0.03475539,
          0.07000978,  0.01799396],
        [-0.04456381, -0.04624762, -0.0172409 , ...,  0.06080531,
         -0.03362306, -

In [38]:
# pretrained_model.load_weights("/root/spark/model/aa_weights.h5", by_name=True)

In [ ]:
# # Check that all of the pretrained weights have been loaded.
# for a, b in zip(pretrained.weights, model.weights):
#     np.testing.assert_allclose(a.numpy(), b.numpy())

# print("\n", "-" * 50)
# model.summary()

In [33]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu", trainable=False)(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu", trainable=False)(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu", trainable=False)(x)
x = layers.GlobalMaxPooling1D()(x)
# x = layers.Dense(128, activation="relu", trainable=False)(x)
# # x = layers.Dropout(0.5, name='dropout_k')(x)
# preds = layers.Dense(2, activation="softmax")(x)
model = keras.Model(int_sequences_input, x)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embed (Embedding)            (None, None, 100)         2556800   
_________________________________________________________________
conv1d_6 (Conv1D)            (None, None, 128)         64128     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, None, 128)         8204

In [43]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
pretrained_model = keras.Model(inputs=int_sequences_input, outputs=x)

model = keras.Sequential([pretrained_model, layers.Dense(128, activation="relu"), layers.Dense(2, activation="softmax")])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 128)               2785024   
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 258       
Total params: 2,801,794
Trainable params: 244,994
Non-trainable params: 2,556,800
_________________________________________________________________


In [46]:
pretrained_model.load_weights("/root/spark/model/aa_weights.h5")

ValueError: You are trying to load a weight file containing 6 layers into a model with 4 layers.

In [45]:
for w in model.weights:
    print(w)

<tf.Variable 'embed/embeddings:0' shape=(25568, 100) dtype=float32, numpy=
array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.13277 , -0.11802 , -0.063487, ...,  0.10199 ,  1.0979  ,
        -0.17951 ],
       [-0.016233,  0.77261 , -0.052126, ...,  0.067194, -0.2468  ,
        -0.12894 ],
       ...,
       [-0.75831 ,  0.42915 , -0.040031, ...,  0.11268 , -0.088691,
        -0.20963 ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ]], dtype=float32)>
<tf.Variable 'conv1d_9/kernel:0' shape=(5, 100, 128) dtype=float32, numpy=
array([[[-0.06583161, -0.02366187,  0.04817189, ..., -0.00501878,
          0.04024842,  0.00257344],
        [ 0.03357482, -0.06890798, -0.00362643, ..., -0.03384611,
          0.04883148, -0.04939078],
        [-0.02353059, -0.00887622, -0.05387938, ...,  0.00444328,
          0.00635561, -

In [28]:
model.compile(
    loss="categorical_crossentropy", optimizer="sgd", metrics=["acc"]
)

In [36]:
model.load_weights('/root/spark/model/keras_model_1_weights.h5', by_name=True)

In [37]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embed (Embedding)            (None, None, 100)         2556800   
_________________________________________________________________
conv1d_6 (Conv1D)            (None, None, 128)         64128     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, None, 128)         8204

In [38]:
from tensorflow.keras import models
from tensorflow.keras import layers

new_model = models.Sequential()
new_model.add(model)
new_model.add(layers.Dense(128, activation="relu"))
# new_model.add(layers.Dropout(0.5))
new_model.add(layers.Dense(2, activation="softmax"))

In [39]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Functional)         (None, 128)               2785024   
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 258       
Total params: 2,801,794
Trainable params: 244,994
Non-trainable params: 2,556,800
_________________________________________________________________


In [40]:
print('model를 동결하기 전 훈련되는 가중치의 수:',
     len(new_model.trainable_weights))
#model.trainable = False
print('model를 동결한 후 훈련되는 가중치의 수:',
     len(new_model.trainable_weights))

model를 동결하기 전 훈련되는 가중치의 수: 10
model를 동결한 후 훈련되는 가중치의 수: 4


In [95]:
model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["acc"])

In [29]:
from tensorflow.keras import optimizers, regularizers

optimizer_conf = optimizers.SGD(lr=0.0001)
opt_conf = optimizers.serialize(optimizer_conf)

In [30]:
from tensorflow.keras.models import model_from_json
from elephas import spark_model, ml_model
from elephas.ml_model import ElephasEstimator

estimator = ElephasEstimator()
estimator.setFeaturesCol("feature")
estimator.setLabelCol("label_index")
estimator.set_keras_model_config(model.to_json())
estimator.set_categorical_labels(True) # one-hot encoding 여부
estimator.set_nb_classes(2)
estimator.set_num_workers(3)
estimator.set_epochs(100)
estimator.set_batch_size(128)
estimator.set_verbosity(1)
estimator.set_validation_split(0.10)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("synchronous")
estimator.set_loss("categorical_crossentropy")
estimator.set_metrics(['acc'])

ElephasEstimator_7e53284c5961

In [50]:
# set_trainable = False
# for layer in model.layers:
#     if layer.name == ""

In [31]:
new_pipeline = Pipeline(stages=[estimator])
# e1에서 그대로 가져오는지는 확인해볼 필요성이 있다.

In [32]:
import time
t1 = time.time()
new_dl = new_pipeline.fit(df_with_vectors)
t2 = time.time() - t1
print(t2)

>>> Fit model
>>> Synchronous training complete.
747.9298045635223


In [33]:
model.save("model_2_ckpt")

INFO:tensorflow:Assets written to: model_2_ckpt/assets


In [34]:
model.save_weights("pretrained_2_ckpt")

In [46]:
new_model.save('/root/spark/model/keras_model_2.h5')

In [47]:
new_model.save_weights('/root/spark/model/keras_model_2_weights.h5')

In [58]:
from elephas.ml_model import ElephasTransformer
from elephas.ml_model import ElephasEstimator
import h5py
from elephas.ml_model import load_ml_transformer

cs_model = new_dl.stages[0]
print(cs_model)

cs_model.save("/root/spark/model/estimator2")

ElephasTransformer_affcc2c7c67e


In [35]:
class TextToSequence2(Transformer):
    def __init__(self):
        super(TextToSequence2, self).__init__()
        
    def _transform(self, df: DataFrame):
        # 우리가 해야할 일은 dataframe을 순서통일해서 text와 label 리스트로 나누는것
        samples = df.select("text").rdd.flatMap(lambda x: x).collect()
        labels = df.select("label").rdd.flatMap(lambda x: x).collect()
        
        # 200의 길이로 상위 2만개의 토큰만 반영
        #vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
        
        x_test = TextToSequence.vectorizer(np.array([[s] for s in samples])).numpy()
        zip_array = list(zip(samples, labels, x_test.tolist()))
        rdd = sc.parallelize(zip_array, 306) #, numSlices=306
        fdf = rdd.toDF(['text','label','feature'])
        
        return fdf
    
# get_tokens 함수과 같은 용도
stage_5 = TextToSequence2()
stage_6 = StringIndexer(inputCol='label', outputCol='label_index')
test_pipeline = Pipeline(stages=[stage_5, stage_6])

In [36]:
preprocessing2 = test_pipeline.fit(second_df)
preprocessed2 = preprocessing2.transform(second_df)
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
df_test = preprocessed2.select(
    preprocessed2["label_index"], 
    list_to_vector_udf(preprocessed2["feature"]).alias("feature")
)

In [37]:
pred_test = new_dl.transform(df_test)

In [38]:
from pyspark.sql.functions import udf
list_to_double = udf(lambda l: l[0], DoubleType())
pred_test2 = pred_test.select(
    pred_test["label_index"], 
    list_to_double(pred_test["prediction"]).alias("prediction2")
)

In [39]:
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l:1.0 if l<0.5 else 0.0 , DoubleType())
pred_test3 = pred_test2.select(
    pred_test2["label_index"], 
    list_to_vector_udf(pred_test2["prediction2"]).alias("rawPrediction")
)

In [40]:
pred_test.select('prediction').show(20, truncate=False)

+-----------------------------------------+
|prediction                               |
+-----------------------------------------+
|[0.46497347950935364, 0.5350266098976135]|
|[0.5289415121078491, 0.47105854749679565]|
|[0.4707801342010498, 0.5292198657989502] |
|[0.4707200825214386, 0.5292799472808838] |
|[0.5110920667648315, 0.48890793323516846]|
|[0.44770410656929016, 0.5522958636283875]|
|[0.4583183825016022, 0.5416815876960754] |
|[0.4523548185825348, 0.5476451516151428] |
|[0.4553672969341278, 0.5446327328681946] |
|[0.48225489258766174, 0.5177451372146606]|
|[0.4694705903530121, 0.5305294394493103] |
|[0.4483574330806732, 0.5516425967216492] |
|[0.43663471937179565, 0.5633652806282043]|
|[0.49765413999557495, 0.502345860004425] |
|[0.4503838121891022, 0.5496161580085754] |
|[0.46125563979148865, 0.5387443900108337]|
|[0.48579877614974976, 0.5142012238502502]|
|[0.5275406241416931, 0.4724594056606293] |
|[0.4848715662956238, 0.5151284337043762] |
|[0.46705684065818787, 0.5329431

In [41]:
pred_test3.show(10)

+-----------+-------------+
|label_index|rawPrediction|
+-----------+-------------+
|        1.0|          1.0|
|        1.0|          0.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          0.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
+-----------+-------------+
only showing top 10 rows



In [42]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="label_index", predictionCol="rawPrediction", metricName="accuracy")
accuracy = evaluator.evaluate(pred_test3)
print(accuracy)

0.6071


In [55]:
def regex_(text):
    # 영어, 숫자, 특수만문자 제외 삭제.
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https):// (?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text = re.sub(pattern, '', text)
    only_english = re.sub('[^ a-zA-Z]', '', text)
    pic_str = 'pic.twitter.com/'
    only_english = only_english.lower()
    
    return only_english
#     if bool(only_english and only_english.strip()) and len(only_english) >= 10:
#         return only_english
#     return False

In [63]:
udf_nlkt = udf(regex_, StringType())
new_df = df2.withColumn("text", udf_nlkt("text"))

In [64]:
df2.show(3)

+--------------------+
|                text|
+--------------------+
|RT @nanox_finance...|
|*/inamin HAHAHAHA...|
|4.  hai yang mau ...|
+--------------------+
only showing top 3 rows



In [65]:
new_df.show(3)

+--------------------+
|                text|
+--------------------+
|rt nanoxfinance a...|
|inamin hahahahaha...|
|  hai yang mau se...|
+--------------------+
only showing top 3 rows



In [66]:
class TextToSequence3(Transformer):
    def __init__(self):
        super(TextToSequence3, self).__init__()
        
    def _transform(self, df: DataFrame):
        # 우리가 해야할 일은 dataframe을 순서통일해서 text와 label 리스트로 나누는것
        samples = df.select("text").rdd.flatMap(lambda x: x).collect()
        
        x_test = TextToSequence.vectorizer(np.array([[s] for s in samples])).numpy()
        zip_array = list(zip(samples, x_test.tolist()))
        rdd = sc.parallelize(zip_array, 306) #, numSlices=306
        fdf = rdd.toDF(['text', 'feature'])
        
        return fdf

In [67]:
stage_t = TextToSequence3()
test_pipeline = Pipeline(stages=[stage_t])

In [68]:
preprocessing2 = test_pipeline.fit(new_df)
preprocessed2 = preprocessing2.transform(new_df)
preprocessed2.show(3)

+--------------------+--------------------+
|                text|             feature|
+--------------------+--------------------+
|rt nanoxfinance a...|[64, 1, 1, 141, 1...|
|inamin hahahahaha...|[1, 1, 0, 0, 0, 0...|
|  hai yang mau se...|[12344, 12604, 1,...|
+--------------------+--------------------+
only showing top 3 rows



In [69]:
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

df_test = preprocessed2.select(
    preprocessed2["text"], 
    list_to_vector_udf(preprocessed2["feature"]).alias("feature")
)

In [70]:
pred_test = new_dl.transform(df_test)

In [71]:
pred_test.select('prediction').show(3, truncate=False)

+-----------------------------------------+
|prediction                               |
+-----------------------------------------+
|[0.42767348885536194, 0.5723265409469604]|
|[0.05208161845803261, 0.9479184150695801]|
|[0.02548149600625038, 0.9745185375213623]|
+-----------------------------------------+
only showing top 3 rows



In [72]:
from pyspark.sql.functions import udf
list_to_double = udf(lambda l: l[0], DoubleType())
pred_test2 = pred_test.select(
    pred_test["text"], 
    list_to_double(pred_test["prediction"]).alias("prediction2")
)

In [73]:
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l:1.0 if l>=0.9 else (0.0 if l< 0.1 else None) , DoubleType())
pred_test3 = pred_test2.select(
    pred_test2["text"], 
    list_to_vector_udf(pred_test2["prediction2"]).alias("label")
)

In [74]:
pred_test3.show(3, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------+-----+
|text                                                                                                                       |label|
+---------------------------------------------------------------------------------------------------------------------------+-----+
|rt nanoxfinance airdrop alert  nanoxfinance have started it last community airdropno referral needed  guaranteed for all pa|null |
|inamin hahahahahahahahahahahaha                                                                                            |0.0  |
|  hai yang mau sewa zoom unlock cheggcek turnitin bisa banget loh sama aku sudah ada  testi ya bisa cek                    |0.0  |
+---------------------------------------------------------------------------------------------------------------------------+-----+
only showing top 3 rows



In [75]:
pred_test3.printSchema()

root
 |-- text: string (nullable = true)
 |-- label: double (nullable = true)



In [76]:
df = pred_test3.filter((col('label') == 0.0) | (col('label') == 1.0))

In [77]:
df.groupBy('label').count().orderBy('count').show()

+-----+------+
|label| count|
+-----+------+
|  1.0|111107|
|  0.0|421952|
+-----+------+



In [78]:
df.toPandas().to_csv("/root/spark/df2.csv")

In [ ]:
def regex_(text):
    # 영어, 숫자, 특수만문자 제외 삭제.
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https):// (?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text = re.sub(pattern, '', text)
    only_english = re.sub('[^ a-zA-Z]', '', text)
    pic_str = 'pic.twitter.com/'
    only_english = only_english.lower()
    
    return only_english
#     if bool(only_english and only_english.strip()) and len(only_english) >= 10:
#         return only_english
#     return False

udf_nlkt = udf(regex_, StringType())
new_df = df1.withColumn("text", udf_nlkt("text"))

df1.show(3)

new_df.show(3)

class TextToSequence3(Transformer):
    def __init__(self):
        super(TextToSequence3, self).__init__()
        
    def _transform(self, df: DataFrame):
        # 우리가 해야할 일은 dataframe을 순서통일해서 text와 label 리스트로 나누는것
        samples = df.select("text").rdd.flatMap(lambda x: x).collect()
        
        x_test = TextToSequence.vectorizer(np.array([[s] for s in samples])).numpy()
        zip_array = list(zip(samples, x_test.tolist()))
        rdd = sc.parallelize(zip_array, 306) #, numSlices=306
        fdf = rdd.toDF(['text', 'feature'])
        
        return fdf

stage_t = TextToSequence3()
test_pipeline = Pipeline(stages=[stage_t])

preprocessing2 = test_pipeline.fit(new_df)
preprocessed2 = preprocessing2.transform(new_df)
preprocessed2.show(3)

list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

df_test = preprocessed2.select(
    preprocessed2["text"], 
    list_to_vector_udf(preprocessed2["feature"]).alias("feature")
)

pred_test = my_dl.transform(df_test)

pred_test.select('prediction').show(3, truncate=False)

from pyspark.sql.functions import udf
list_to_double = udf(lambda l: l[0], DoubleType())
pred_test2 = pred_test.select(
    pred_test["text"], 
    list_to_double(pred_test["prediction"]).alias("prediction2")
)

from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l:1.0 if l>=0.9 else (0.0 if l< 0.1 else None) , DoubleType())
pred_test3 = pred_test2.select(
    pred_test2["text"], 
    list_to_vector_udf(pred_test2["prediction2"]).alias("label")
)

pred_test3.show(3, truncate=False)

pred_test3.printSchema()

df = pred_test3.filter((col('label') == 0.0) | (col('label') == 1.0))

df.groupBy('label').count().orderBy('count').show()

df.toPandas().to_csv("/root/spark/df1.csv")

# 모델 학습

In [ ]:
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["acc"])

## 1. SparkModel

In [35]:
x_train = df_with_vectors.select("feature").rdd.flatMap(lambda x: x).collect()
y_train = df_with_vectors.select("label_index").rdd.flatMap(lambda x: x).collect()

In [37]:
from elephas.utils.rdd_utils import to_simple_rdd
rdd = to_simple_rdd(sc, x_train, y_train)

In [38]:
from elephas.spark_model import SparkModel

spark_model = SparkModel(model, frequency='epoch', mode='asynchronous')
spark_model.fit(rdd, epochs=20, batch_size=32, verbose=0, validation_split=0.1)

>>> Fit model
 * Serving Flask app 'elephas.parameter.server' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://172.28.0.1:4000/ (Press CTRL+C to quit)


>>> Initialize workers
>>> Distribute load


172.28.0.1 - - [17/Jan/2022 04:44:56] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [17/Jan/2022 04:44:56] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [17/Jan/2022 04:44:56] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [17/Jan/2022 04:44:57] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [17/Jan/2022 04:44:57] "GET /parameters HTTP/1.1" 200 -
172.28.0.2 - - [17/Jan/2022 04:44:57] "GET /parameters HTTP/1.1" 200 -
172.28.0.2 - - [17/Jan/2022 04:44:57] "GET /parameters HTTP/1.1" 200 -
172.28.0.3 - - [17/Jan/2022 04:44:57] "GET /parameters HTTP/1.1" 200 -
172.28.0.2 - - [17/Jan/2022 04:44:57] "GET /parameters HTTP/1.1" 200 -
172.28.0.3 - - [17/Jan/2022 04:44:57] "GET /parameters HTTP/1.1" 200 -
172.28.0.2 - - [17/Jan/2022 04:44:57] "GET /parameters HTTP/1.1" 200 -
172.28.0.2 - - [17/Jan/2022 04:44:57] "GET /parameters HTTP/1.1" 200 -
172.28.0.3 - - [17/Jan/2022 04:44:57] "GET /parameters HTTP/1.1" 200 -
172.28.0.3 - - [17/Jan/2022 04:44:58] "GET /parameters HTTP/1.1" 200 -
172.28

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 24.0 failed 4 times, most recent failure: Lost task 7.3 in stage 24.0 (TID 6203, kafka1, executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/root/spark/python/pyspark/worker.py", line 377, in main
    process()
  File "/root/spark/python/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/root/spark/python/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/lib/python3.6/dist-packages/elephas/worker.py", line 108, in train
    self.model.fit(x_train, y_train, **self.train_config)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py", line 1095, in fit
    tmp_logs = self.train_function(it

172.28.0.1 - - [17/Jan/2022 04:45:10] "GET /parameters HTTP/1.1" 200 -
172.28.0.2 - - [17/Jan/2022 04:45:10] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [17/Jan/2022 04:45:10] "GET /parameters HTTP/1.1" 200 -
172.28.0.2 - - [17/Jan/2022 04:45:10] "GET /parameters HTTP/1.1" 200 -
172.28.0.3 - - [17/Jan/2022 04:45:10] "GET /parameters HTTP/1.1" 200 -
172.28.0.3 - - [17/Jan/2022 04:45:10] "GET /parameters HTTP/1.1" 200 -
172.28.0.2 - - [17/Jan/2022 04:45:10] "GET /parameters HTTP/1.1" 200 -


## 2. ElephasEstimator (v)

In [25]:
from tensorflow.keras import optimizers, regularizers

optimizer_conf = optimizers.SGD(lr=0.001) #0.0001
opt_conf = optimizers.serialize(optimizer_conf)

In [26]:
from tensorflow.keras.models import model_from_json
from elephas import spark_model, ml_model
from elephas.ml_model import ElephasEstimator

estimator = ElephasEstimator()
estimator.setFeaturesCol("text")
estimator.setLabelCol("label_index")
estimator.set_keras_model_config(pretrained.to_json())
estimator.set_categorical_labels(True) # one-hot encoding 여부
estimator.set_nb_classes(2)
estimator.set_num_workers(3)
estimator.set_epochs(100)
estimator.set_batch_size(128)
estimator.set_verbosity(1)
estimator.set_validation_split(0.10)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("synchronous")
estimator.set_loss("categorical_crossentropy")
estimator.set_metrics(['acc'])

ElephasEstimator_78731217fd3f

In [27]:
dl_pipeline = Pipeline(stages=[estimator])

In [28]:
df_with_vectors.printSchema()

root
 |-- text: string (nullable = true)
 |-- label_index: double (nullable = false)
 |-- feature: vector (nullable = true)



In [29]:
import time
t1 = time.time()
my_dl = dl_pipeline.fit(df_with_vectors)
t2 = time.time() - t1
print(t2)

>>> Fit model


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 12 in stage 21.0 failed 4 times, most recent failure: Lost task 12.3 in stage 21.0 (TID 4473, kafka1, executor 2): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/root/spark/python/pyspark/worker.py", line 377, in main
    process()
  File "/root/spark/python/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/root/spark/python/pyspark/serializers.py", line 352, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/root/spark/python/pyspark/serializers.py", line 142, in dump_stream
    for obj in iterator:
  File "/root/spark/python/pyspark/serializers.py", line 341, in _batched
    for item in iterator:
  File "/root/spark/python/pyspark/util.py", line 99, in wrapper
    re

In [ ]:
# 1)

In [28]:
pretrained.save("model_1_ckpt")

INFO:tensorflow:Assets written to: model_1_ckpt/assets


In [29]:
pretrained.save_weights("pretrained_1_ckpt")

In [46]:
model.save('/root/spark/model/keras_model_1')

In [47]:
model.save_weights('/root/spark/model/keras_model_1_weights')

In [ ]:
# 2)

In [67]:
new_model.save('/root/spark/model/new_model.h5')

In [68]:
new_model.save_weights('/root/spark/model/new_model_weights.h5')

In [73]:
# 1. Estimator를 저장하기

In [ ]:
from elephas.ml_model import load_ml_estimator

In [ ]:
estimator.save("/root/spark/model/v1.h5")

In [ ]:
#new_estimator = ElephasEstimator()
new_estimator = load_ml_estimator("/root/spark/model/v1.h5")

In [ ]:
# 2. Transformer를 저장하기

In [48]:
from elephas.ml_model import ElephasTransformer
from elephas.ml_model import ElephasEstimator
import h5py
from elephas.ml_model import load_ml_transformer

In [49]:
cs_model = my_dl.stages[0]
print(cs_model)

ElephasTransformer_555fc48a907f


In [50]:
cs_model.save("/root/spark/model/estimator1")

In [16]:
new_transformer = load_ml_transformer("/root/spark/model/estimator")

NameError: name 'load_ml_transformer' is not defined

In [ ]:
# 3. Model을 저장하기

In [46]:
model.save('/root/spark/model/keras_model.h5')

In [29]:
loaded_model = tf.keras.models.load_model('/root/spark/model/keras_model.h5')

In [ ]:
# loaded_model을 fix하고 untrainable하게 몇개의 layer를 trainable 만든다음 define the estimator again

In [ ]:
# pipeline에 넣어서 fit again

In [55]:
new_estimator = ElephasEstimator()
new_estimator.set_keras_model_config(loaded_model.to_json())
new_estimator.setFeaturesCol("feature")
new_estimator.setLabelCol("label_index")

ElephasEstimator_f5cdbf85529a

In [52]:
new_pipeline = Pipeline(stages=[new_estimator])

In [53]:
df_with_vectors.columns

['label_index', 'feature']

In [56]:
new_transformer = new_estimator.transform(df_with_vectors)

AttributeError: 'ElephasEstimator' object has no attribute 'transform'

In [ ]:
# 4. Transformer를 저장하기

In [ ]:
writer = indexerModel._call_java("write")
writer.save("indexerModel")

## 기존 Transformer를 이용해서 예측

In [51]:
class TextToSequence2(Transformer):
    def __init__(self):
        super(TextToSequence2, self).__init__()
        
    def _transform(self, df: DataFrame):
        # 우리가 해야할 일은 dataframe을 순서통일해서 text와 label 리스트로 나누는것
        samples = df.select("text").rdd.flatMap(lambda x: x).collect()
        labels = df.select("label").rdd.flatMap(lambda x: x).collect()
        
        # 200의 길이로 상위 2만개의 토큰만 반영
        #vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
        
        x_test = TextToSequence.vectorizer(np.array([[s] for s in samples])).numpy()
        zip_array = list(zip(samples, labels, x_test.tolist()))
        rdd = sc.parallelize(zip_array, 306) #, numSlices=306
        fdf = rdd.toDF(['text','label','feature'])
        
        return fdf
    
# get_tokens 함수과 같은 용도
stage_5 = TextToSequence2()
stage_6 = StringIndexer(inputCol='label', outputCol='label_index')
test_pipeline = Pipeline(stages=[stage_5, stage_6])


In [52]:
preprocessing2 = test_pipeline.fit(second_df)
preprocessed2 = preprocessing2.transform(second_df)

df_test = preprocessed2.select(
    preprocessed2["label_index"], 
    list_to_vector_udf(preprocessed2["feature"]).alias("feature")
)

In [53]:
df_test.show(5)

+-----------+--------------------+
|label_index|             feature|
+-----------+--------------------+
|        1.0|[9706.0,10047.0,7...|
|        1.0|[1683.0,16.0,1208...|
|        1.0|[8767.0,1.0,1234....|
|        1.0|[1318.0,1185.0,11...|
|        1.0|[7658.0,1.0,1302....|
+-----------+--------------------+
only showing top 5 rows



In [54]:
pred_test = my_dl.transform(df_test)

In [55]:
pred_test.select('prediction').show(3, truncate=False)

+-----------------------------------------+
|prediction                               |
+-----------------------------------------+
|[0.00904042087495327, 0.990959644317627] |
|[0.7892279624938965, 0.21077196300029755]|
|[0.8867055773735046, 0.11329440027475357]|
+-----------------------------------------+
only showing top 3 rows



In [56]:
from pyspark.sql.functions import udf
list_to_double = udf(lambda l: l[0], DoubleType())
pred_test2 = pred_test.select(
    pred_test["label_index"], 
    list_to_double(pred_test["prediction"]).alias("prediction2")
)

In [57]:
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l:1.0 if l<0.5 else 0.0 , DoubleType())
pred_test3 = pred_test2.select(
    pred_test2["label_index"], 
    list_to_vector_udf(pred_test2["prediction2"]).alias("rawPrediction")
)

In [58]:
pred_test3.show(20)

+-----------+-------------+
|label_index|rawPrediction|
+-----------+-------------+
|        1.0|          1.0|
|        1.0|          0.0|
|        1.0|          0.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          0.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          0.0|
+-----------+-------------+
only showing top 20 rows



In [ ]:
pred_test3.groupBy('rawPrediction').count().orderBy('count').show()

In [36]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="label_index", predictionCol="rawPrediction", metricName="accuracy")
accuracy = evaluator.evaluate(pred_test3)
print(accuracy)

0.87435


In [ ]:
#########################################################################################

In [59]:
def regex_(text):
    # 영어, 숫자, 특수만문자 제외 삭제.
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https):// (?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text = re.sub(pattern, '', text)
    only_english = re.sub('[^ a-zA-Z]', '', text)
    pic_str = 'pic.twitter.com/'
    only_english = only_english.lower()
    
    return only_english
#     if bool(only_english and only_english.strip()) and len(only_english) >= 10:
#         return only_english
#     return False

In [63]:
udf_nlkt = udf(regex_, StringType())
new_df = df1.withColumn("text", udf_nlkt("text"))

In [64]:
df1.show(3)

+--------------------+
|                text|
+--------------------+
|RT @nanox_finance...|
|*/inamin HAHAHAHA...|
|4.  hai yang mau ...|
+--------------------+
only showing top 3 rows



In [65]:
new_df.show(3)

+--------------------+
|                text|
+--------------------+
|rt nanoxfinance a...|
|inamin hahahahaha...|
|  hai yang mau se...|
+--------------------+
only showing top 3 rows



In [66]:
class TextToSequence3(Transformer):
    def __init__(self):
        super(TextToSequence3, self).__init__()
        
    def _transform(self, df: DataFrame):
        # 우리가 해야할 일은 dataframe을 순서통일해서 text와 label 리스트로 나누는것
        samples = df.select("text").rdd.flatMap(lambda x: x).collect()
        
        x_test = TextToSequence.vectorizer(np.array([[s] for s in samples])).numpy()
        zip_array = list(zip(samples, x_test.tolist()))
        rdd = sc.parallelize(zip_array, 306) #, numSlices=306
        fdf = rdd.toDF(['text', 'feature'])
        
        return fdf

In [67]:
stage_t = TextToSequence3()
test_pipeline = Pipeline(stages=[stage_t])

In [68]:
preprocessing2 = test_pipeline.fit(new_df)
preprocessed2 = preprocessing2.transform(new_df)
preprocessed2.show(3)

+--------------------+--------------------+
|                text|             feature|
+--------------------+--------------------+
|rt nanoxfinance a...|[64, 1, 1, 141, 1...|
|inamin hahahahaha...|[1, 1, 0, 0, 0, 0...|
|  hai yang mau se...|[12344, 12604, 1,...|
+--------------------+--------------------+
only showing top 3 rows



In [69]:
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

df_test = preprocessed2.select(
    preprocessed2["text"], 
    list_to_vector_udf(preprocessed2["feature"]).alias("feature")
)

In [70]:
pred_test = my_dl.transform(df_test)

In [71]:
pred_test.select('prediction').show(3, truncate=False)

+-----------------------------------------+
|prediction                               |
+-----------------------------------------+
|[0.42767348885536194, 0.5723265409469604]|
|[0.05208161845803261, 0.9479184150695801]|
|[0.02548149600625038, 0.9745185375213623]|
+-----------------------------------------+
only showing top 3 rows



In [72]:
from pyspark.sql.functions import udf
list_to_double = udf(lambda l: l[0], DoubleType())
pred_test2 = pred_test.select(
    pred_test["text"], 
    list_to_double(pred_test["prediction"]).alias("prediction2")
)

In [73]:
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l:1.0 if l>=0.9 else (0.0 if l< 0.1 else None) , DoubleType())
pred_test3 = pred_test2.select(
    pred_test2["text"], 
    list_to_vector_udf(pred_test2["prediction2"]).alias("label")
)

In [74]:
pred_test3.show(3, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------+-----+
|text                                                                                                                       |label|
+---------------------------------------------------------------------------------------------------------------------------+-----+
|rt nanoxfinance airdrop alert  nanoxfinance have started it last community airdropno referral needed  guaranteed for all pa|null |
|inamin hahahahahahahahahahahaha                                                                                            |0.0  |
|  hai yang mau sewa zoom unlock cheggcek turnitin bisa banget loh sama aku sudah ada  testi ya bisa cek                    |0.0  |
+---------------------------------------------------------------------------------------------------------------------------+-----+
only showing top 3 rows



In [75]:
pred_test3.printSchema()

root
 |-- text: string (nullable = true)
 |-- label: double (nullable = true)



In [76]:
df = pred_test3.filter((col('label') == 0.0) | (col('label') == 1.0))

In [77]:
df.groupBy('label').count().orderBy('count').show()

+-----+------+
|label| count|
+-----+------+
|  1.0|111107|
|  0.0|421952|
+-----+------+



In [78]:
df.toPandas().to_csv("/root/spark/df1.csv")

In [79]:
# Import data types
from pyspark.sql.types import *

schema = StructType(
   [
    StructField('text', StringType(), False),
    StructField('label', DoubleType(), False)
   ]
)

In [90]:
first = spark.read.format("csv").option("header", True).schema(schema).load("hdfs:///root/spark/first.csv")

In [91]:
first.count()

0

In [ ]:
first.show(3)

In [ ]:
pred_test3.show(3)

In [ ]:
# pred_test3을 저장하기

In [ ]:
#########################################################################################

## 저장된 Model을 불러와서 예측

In [17]:
from elephas.ml_model import ElephasTransformer
import h5py
from elephas.ml_model import load_ml_transformer

In [18]:
new_transformer = load_ml_transformer("/root/spark/model/estimator")

In [62]:
pred_test_ = new_transformer.transform(df_test)

In [63]:
pred_test_.show(5)

+-----------+--------------------+--------------------+
|label_index|             feature|          prediction|
+-----------+--------------------+--------------------+
|        1.0|[42.0,3.0,540.0,1...|[0.49246680736541...|
|        1.0|[177.0,1710.0,336...|[0.49246680736541...|
|        1.0|[4390.0,837.0,487...|[0.49246680736541...|
|        1.0|[3525.0,1328.0,12...|[0.49246680736541...|
|        1.0|[121.0,4969.0,148...|[0.49246680736541...|
+-----------+--------------------+--------------------+
only showing top 5 rows



In [66]:
pred_test.show(5)

+-----------+--------------------+--------------------+
|label_index|             feature|          prediction|
+-----------+--------------------+--------------------+
|        1.0|[42.0,3.0,540.0,1...|[0.48319944739341...|
|        1.0|[177.0,1710.0,336...|[0.48319944739341...|
|        1.0|[4390.0,837.0,487...|[0.48319944739341...|
|        1.0|[3525.0,1328.0,12...|[0.48319944739341...|
|        1.0|[121.0,4969.0,148...|[0.48319944739341...|
+-----------+--------------------+--------------------+
only showing top 5 rows



In [55]:
def prediction(text):    
    tmp = list()
    tmp.append(str(text))
    y_prob = new_model.predict([tmp])
    y_classes = y_prob.argmax(axis=-1)
    prediction = y_classes.tolist()[0]
    return prediction

In [56]:
udf_model = udf(prediction, StringType())
new_df = label_df.withColumn("prediction", udf_model("text"))
# new_df = new_df.drop("text")

Traceback (most recent call last):
  File "/root/spark/python/pyspark/serializers.py", line 597, in dumps
    return cloudpickle.dumps(obj, 2)
  File "/root/spark/python/pyspark/cloudpickle.py", line 863, in dumps
    cp.dump(obj)
  File "/root/spark/python/pyspark/cloudpickle.py", line 260, in dump
    return Pickler.dump(self, obj)
  File "/usr/lib/python3.6/pickle.py", line 409, in dump
    self.save(obj)
  File "/usr/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "/usr/lib/python3.6/pickle.py", line 736, in save_tuple
    save(element)
  File "/usr/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "/root/spark/python/pyspark/cloudpickle.py", line 400, in save_function
    self.save_function_tuple(obj)
  File "/root/spark/python/pyspark/cloudpickle.py", line 549, in save_function_tuple
    save(state)
  File "/usr/lib/python3.6/pickle.py", line 476, in save
    f(

PicklingError: Could not serialize object: TypeError: can't pickle weakref objects

In [62]:
new_df.show(3)

+--------------------+-----+-----------+--------------------+
|                text|label|label_index|          prediction|
+--------------------+-----+-----------+--------------------+
|how to run spider...|    1|        1.0|[[how to run spid...|
|wordpress magic f...|    1|        1.0|[[wordpress magic...|
|announcing apache...|    1|        1.0|[[announcing apac...|
+--------------------+-----+-----------+--------------------+
only showing top 3 rows



In [ ]:
pred_test = my_dl.transform(second_df)

In [ ]:
from pyspark.sql.functions import udf
list_to_double = udf(lambda l: l[0], DoubleType())
pred_test = pred_test.select(
    pred_test["label_index"], 
    list_to_double(pred_test["prediction"]).alias("prediction2")
)

In [ ]:
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l:1.0 if l<0.5 else 0.0 , DoubleType())
pred_test = pred_test.select(
    pred_test["label_index"], 
    list_to_vector_udf(pred_test["prediction2"]).alias("rawPrediction")
)

In [ ]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="label_index", predictionCol="rawPrediction", metricName="accuracy")
accuracy = evaluator.evaluate(pred_test)
print(accuracy)